# 初期セットアップを行う

このタスクでは、リサーチフロー機能利用のための準備を行います。  
上から順番に全てのセルを実行してください。

研究フロートップページに戻る場合は[こちら](../../base_FLOW.ipynb)。新規タブで開きます。  

## 0. 研究リポジトリ名を確認する  
以下のセルを実行すると、この研究実行環境で操作する研究リポジトリ名を確認できます。  

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
utils.show_name('green')

## 1. ユーザーの認証を行う

この手順では、GIN-forkと同期するための準備として、あなたのユーザ情報をシステムに認証させる手続きを行います。  
以下のセルを実行し、画面の表示に沿ってGIN-forkに登録したユーザー名、パスワード、メールアドレスを入力してください。  
各項目入力後、Enterキーを押下してください。  

In [ ]:
import os
import time
import json
import getpass
import requests

from IPython.display import clear_output
from requests.auth import HTTPBasicAuth
from http import HTTPStatus

tokens = []
access_token = {}

# 正常に認証が終わるまで繰り返し
path_flow_root = '/home/jovyan/WORKFLOWS/FLOW/'
os.chdir(path_flow_root)
from util.scripts import utils
tokens, access_token, name, email = utils.verify_GIN_user()

os.chdir(os.environ['HOME'])
!git config --global user.name $name
!git config --global user.email $email
!cp ~/.gitconfig ~/WORKFLOWS/PACKAGE/.gitconfig

# access_tokenを書き出す
token_dict = {"ginfork_token": access_token['sha1']}
with open('/home/jovyan/.token.json', 'w') as f:
    json.dump(token_dict, f, indent=4)

clear_output()
print("認証が正常に完了しました。次の手順へお進みください。")

## 2. データ同期のための設定をする

この手順では、今の実行環境とGIN-forkのリポジトリでデータの同期をとるための準備をします。  
以下を実行することで、システムがデータ同期の準備の手続きを行います。

In [ ]:
%%bash
#!/bin/bash
if [ ! -e ~/.ssh/id_ed25519 ]; then
    # 鍵ペアが無ければ作成
    ssh-keygen -t ed25519 -N "" -f ~/.ssh/id_ed25519
fi
if [ ! -d ~/.datalad/ ]; then
    # Dataladのデータセットでなければデータセット化する
    datalad create --force /home/jovyan
fi

In [ ]:
# 公開鍵アップロード
# refs: https://github.com/gogs/docs-api/blob/master/Users/Public%20Keys.md#create-a-public-key
import os
import requests
import time
from http import HTTPStatus

import json

pubkey = !cat ~/.ssh/id_ed25519.pub

os.chdir(path_flow_root)
from util.scripts import utils

# 以下の認証の手順で用いるGINのドメイン名等をパラメタファイルから取得する
params = {}
with open(utils.fetch_param_file_path(), mode='r') as f:
    params = json.load(f)


# 認証時に取得したトークンを使ってPOSTリクエスト
response = requests.post(
                params['siblings']['ginHttp']+'/api/v1/user/keys?token=' + access_token['sha1'],
                data={
                    "title": "system-generated-"+str(time.time()),
                    "key": pubkey[0]
                })
msg = response.json()

if response.status_code == HTTPStatus.CREATED:
    print('公開鍵の用意が出来ました。')
elif msg['message'] == 'Key content has been used as non-deploy key':
    print('すでに公開鍵の用意が完了しています。')

In [ ]:
import os
import subprocess
from datalad import api
from IPython.display import clear_output

# sibling url をsshに変更する
os.chdir(os.environ['HOME'])
http_url = subprocess.getoutput('git config --get remote.origin.url')
ssh_url = http_url.replace(params['siblings']['ginHttp'], params['siblings']['ginSsh'])
    
# siblingsにGINを登録する
sibling = !datalad siblings -s gin
for item in sibling:
    if 'unknown sibling name' in item:
        api.siblings(action='add', name='gin', url=ssh_url)
    else:
        pass

clear_output()
print('SSH接続の準備が完了しました。')

## 3. リポジトリ内のファイルを更新する

研究リポジトリの以下のファイルを更新します。  
- .gitignoreの新規作成：当実行環境からGIN-forkに同期しないフォルダやファイルを定義します。
- README.mdの更新：当実行環境に遷移するためのリンクを追記します。

### 3-1. Git管理対象外ファイルを.gitignoreで設定する

In [ ]:
%%sh
#!/bin/bash

if [ ! -f ~/.gitignore ]; then  # .gitignoreファイルがなければ作成

echo "
/.cache/
/.conda/
/.config/
/.ipython/
/.local/
/.tmp/
/.bashrc
/.bash_logout
/.profile
/.netrc
.ipynb_checkpoints/
.fonts/
.jupyter/
.npm/
.ssh/
.jupyter-server-log.txt
.gitconfig
.repository_id
.token.json
.github
__pycache__/
" >> ~/.gitignore

fi

### 3-2. READMEに実行環境へのリンクを追加する

In [ ]:
import os
import urllib
from IPython.display import clear_output

os.chdir(os.environ['HOME'])

# READMEに遷移リンクがあれば削除してから、新しく実行環境のリンクを追記する
path = 'README.md'
s = ''
with open(path, 'r') as f:
    s = f.read()
    s = s[:s.find('## 研究実行環境にアクセスしたい場合')]

with open(path, 'w') as f:
    f.write(s)

with open(path, 'a', newline='\n') as f:
    f.write("\n## 研究実行環境にアクセスしたい場合")
    f.write("\n実行環境へ遷移する場合は以下のリンクをクリックしてください。別の環境へ移るため、新しいタブまたはウィンドウで開くことを推奨します。<br>")
    f.write("\nhttps://jupyter.cs.rcos.nii.ac.jp" + os.environ["JUPYTERHUB_SERVICE_PREFIX"] + "notebooks/WORKFLOWS/base_FLOW.ipynb<br>")
    f.write("\n\n上記リンクからアクセスできない場合は以下のリンクから研究実行環境を再構築してください。<br>")
    f.write("\n再構築後は初期セットアップを再度行ってください。<br>")
    f.write("\nhttps://binder.cs.rcos.nii.ac.jp/v2/git/" + urllib.parse.quote(http_url, safe='') + "/HEAD?filepath=WORKFLOWS/base_FLOW.ipynb")

clear_output()
print('READMEに実行環境へのリンクを追加しました。')

## 4. 本タスクの実行結果をGIN-forkに同期する

ここまでの内容を保存し、GIN-forkに同期します。  
以下のセルを実行してください。

In [ ]:
# ノートブックを保存する
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import os
os.chdir(path_flow_root)
from util.scripts import utils

# SSHホスト（＝GIN）を信頼する設定
utils.config_GIN(ginHttp = params['siblings']['ginHttp'])

# S3にあるデータをGIN-forkに同期しないための設定
!git annex untrust here
!git annex --force trust web

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[GINへの同期の失敗を解消する。](../../conflict_helper.ipynb)を参照して、競合を解消してください。

In [ ]:
import os
os.chdir(path_flow_root)
from util.scripts import utils
os.chdir(os.environ['HOME'])

git_path = ['/home/jovyan/.gitignore', '/home/jovyan/WORKFLOWS', '/home/jovyan/README.md', '/home/jovyan/maDMP.ipynb']

is_ok = utils.syncs_with_repo(git_path=git_path, gitannex_path=[], gitannex_files=[], message='[GIN] 研究リポジトリ初期セットアップを完了', get_paths=[])

## 5. フロー図を更新する

フロー図に初期セットアップが実行済みであることを反映します。

In [ ]:
import os
os.chdir(os.environ['HOME'])

ex_path = 'WORKFLOWS/EX-WORKFLOWS/images/notebooks.diag'
re_path = 'WORKFLOWS/images/notebooks.diag'

with open('.gitignore', 'r') as f:
    has_re_diag = False
    has_ex_diag = False
    for line in f:
        if ex_path in line:
            has_ex_diag = True
        if re_path in line:
            has_re_diag = True
    
    if not has_re_diag:
        !echo $re_path >> ./.gitignore
    if not has_ex_diag:
        !echo $ex_path >> ./.gitignore


# フロー図を更新する
find = '"base_required_every_time"[fontsize = 10];'
replace = '"base_required_every_time"[numbered = 済, fontsize = 10];'

with open(re_path, 'r') as f:
    s = f.read()

with open(re_path, 'w') as f:
    s = s.replace(find, replace)
    f.write(s)

## 6. 研究フロートップページに遷移する

以下のセルを実行し、表示されるリンクをクリックして研究フロートップページに戻ってください。  

In [ ]:
from IPython.display import display, HTML, Javascript
display(HTML("<a href='../../base_FLOW.ipynb'>研究フロートップページに遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))